In [1]:
#Default
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def reset(df):
    cols = df.columns
    return df.reset_index()[cols]
def print_counts(df):
    cols = df.columns
    for each in cols:
        print(each)
        print(df[each].value_counts())
        print('______________________________________')
# ~
#Default Ending

import os
from tqdm import tqdm

print('OK !')

OK !


In [2]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import DensePassageRetriever, EmbeddingRetriever, FARMReader, EntityExtractor
from haystack.document_stores import ElasticsearchDocumentStore

print('OK !')

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


OK !


In [3]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor

preprocessor_100 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)

preprocessor_200 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
)

preprocessor_300 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=300,
    split_respect_sentence_boundary=True,
)


print('OK !')

OK !


# Set preprocessor 

In [4]:
preprocessor = preprocessor_100

# Process Raw PDF

In [5]:
root = 'International Regulator'

file_list = []
for path, subdirs, files in os.walk(root):
    for name in files:
#         print(os.path.join(path, name))
        file_list.append(os.path.join(path, name))
    
len(file_list)
# file_list

file_list = [each for each in file_list if '.pdf' in each or '.PDF' in each]

len(file_list)

384

382

In [6]:
converter = PDFToTextConverter(remove_numeric_tables=False, valid_languages = ["en"])

pdftotext version 4.03 [www.xpdfreader.com]
Copyright 1996-2021 Glyph & Cog, LLC


# Detect ENG Language

In [7]:
from langdetect import detect

print('OK !')

OK !


In [8]:
doc_list = []
error_list = []

for file_name in tqdm(file_list):
    try:
        doc = converter.convert(file_path = file_name, meta = None)
        docs = preprocessor.process(doc[0])
        lang_list = []
        for i in range(len(docs)): 
            try:
                temp_detect = detect(docs[i]['content'])
                lang_list.append(temp_detect)
            except:
                pass
        en_df = pd.DataFrame(lang_list)[0].value_counts(normalize = True).reset_index()
        en_score = float(en_df[en_df['index'] == 'en'][0].values[0])

        if float(en_score) < 0.9:
            error_list.append(file_name)
            print('Error:', file_name)
    except:
        error_list.append(file_name)
        print('Error:', file_name)
        pass

 18%|███████▏                                | 69/382 [00:28<00:36,  8.57it/s]WARNING - haystack.nodes.file_converter.pdf -  The language for International Regulator/25-MAS/other/SG_289.pdf is not one of ['en']. The file may not have been decoded in the correct text format.


Error: International Regulator/25-MAS/other/SG_289.pdf


 29%|███████████▍                           | 112/382 [00:40<01:08,  3.92it/s]Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
 34%|█████████████▏                         | 129/382 [01:05<01:35,  2.65it/s]Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't read xref table
Syntax Warning: PDF file is damaged - attempting to reconstruct xref table...
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
WARNING - haystack.nodes.file_converter.pdf -  The language for International Regulator/25-MAS/other/SG_191.pdf is not one of ['en']. The file may not have been decoded in the correct text format.
 34%|█████████████▎                         | 131/382 [01:05<01:05,  3.82it/s]

Error: International Regulator/25-MAS/other/SG_191.pdf


 38%|██████████████▉                        | 146/382 [01:10<01:10,  3.36it/s]Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't read xref table
Syntax Warning: PDF file is damaged - attempting to reconstruct xref table...
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
WARNING - haystack.nodes.file_converter.pdf -  The language for International Regulator/25-MAS/other/SG_192.pdf is not one of ['en']. The file may not have been decoded in the correct text format.


...

 73%|████████████████████████████▌          | 280/382 [01:50<00:42,  2.39it/s]Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't read xref table
Syntax Warning: PDF file is damaged - attempting to reconstruct xref table...
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
WARNING - haystack.nodes.file_converter.pdf -  The language for International Regulator/25-MAS/other/SG_294.pdf is not one of ['en']. The file may not have been decoded in the correct text format.
 74%|████████████████████████████▊          | 282/382 [01:50<00:31,  3.22it/s]

Error: International Regulator/25-MAS/other/SG_294.pdf


 93%|████████████████████████████████████▍  | 357/382 [02:28<00:17,  1.40it/s]

Error: International Regulator/03-BCBS/risk/Ripple effects of monetary policy.pdf


100%|███████████████████████████████████████| 382/382 [02:48<00:00,  2.27it/s]


In [9]:
for i, each in enumerate(error_list):
    print(str(i) + ':', each)

0: International Regulator/25-MAS/other/SG_289.pdf
1: International Regulator/25-MAS/other/SG_191.pdf
2: International Regulator/25-MAS/other/SG_192.pdf
3: International Regulator/25-MAS/other/SG_193.pdf
4: International Regulator/25-MAS/other/SG_346.pdf
5: International Regulator/25-MAS/other/SG_235.pdf
6: International Regulator/25-MAS/other/SG_290.pdf
7: International Regulator/25-MAS/other/SG_246.pdf
8: International Regulator/25-MAS/other/SG_241.pdf
9: International Regulator/25-MAS/other/SG_294.pdf
10: International Regulator/03-BCBS/risk/Ripple effects of monetary policy.pdf
